In [17]:
# IMPORTAÇÕES
# ==========================================

# Bibliotecas para manipulação do sistema e ambiente de aprendizado por reforço
import os

# Bibliotecas para operações matemáticas e aleatoriedade
import numpy as np
import random
import logging  # Para registro de mensagens e depuração
import itertools  # Para geração de combinações e permutações
import statistics
import cv2
import csv
import math
import heapq
import json


# Biblioteca para manipulações de DataFrame
import pandas as pd

# Biblioteca para operações de Deep Learning com TensorFlow
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, Flatten, Dense, InputLayer, MultiHeadAttention, LayerNormalization, Reshape, MultiHeadAttention, GlobalAveragePooling2D, TimeDistributed, MaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam  # Otimizador para ajuste dos pesos da rede neural
from tensorflow.keras.losses import MeanSquaredError, Huber
from tensorflow.keras.regularizers import l2  # Regularização L2 para controle de overfitting
from tensorflow.keras.models import load_model, Model  # Carregar modelos pré-treinados, 



from tensorflow.keras.layers import (InputLayer, Conv2D, BatchNormalization, Flatten, 
                                     Dense, Dropout, Activation, Concatenate, Multiply, Layer)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError, Huber
import tensorflow.keras.backend as K


# Verificar dispositivos físicos disponíveis (por exemplo, GPU)
#device_lib.list_local_devices()
tf.config.list_physical_devices('GPU')



[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### CLASS
----------

In [18]:
#Class: Agents
# ==========================================

class Agent:
    def __init__(self, x, y, env, breed, channel):
        self.x = x
        self.y = y
        self.env = env
        self.breed = breed
        self.channel = channel
        self.is_alive = True
        self.is_done = False
        self.current_target = None
        self.current_ally = None
        self.last_coord = self.x, self.y
        self.kid = False

        # Category         | Breed | Color  |     Channel      | Color - Att | Channel - Att
        # ---------------------------------------------------------------------------
        # Predator         |   0   | Red    |    [1, 0, 0]     |   Yellow     | [1, 1, 0]
        # Vegetation       |   1   | Green  |    [0, 1, 0]     |  --------    | ---------
        # Prey             |   2   | Blue   |    [0, 0, 1]     |   Cyan       | [0, 1, 1]
        # Outside Grid     |   -   | White  |    [1, 1, 1]     |  --------    | ---------
        # Empty Grid Cell  |   -   | Black  |    [0, 0, 0]     |  --------    | ---------
        # Reserved Training|   -   | Gray   | [0.5, 0.5, 0.5]  |  --------    | ---------

    def reset(self, env):
        self.x, self.y = env.new_position()
        self.is_alive = True
        self.target = None
        self.is_done = False
        self.current_target = None
        self.current_ally = None
        
        # Restaurar o `channel` para a cor padrão com base no `breed`
        if self.breed == 0:  # Predador
            self.channel = [1.0, 0.0, 0.0]
        elif self.breed == 2:  # Presa
            self.channel = [0.0, 0.0, 1.0]

    def step(self, action):
        penalty = self.env.move_agent(self, action)
        done, reward, ler = self.check_goal()
        state = self.env.render_agent(self)

        return state, reward + penalty, done, ler
    
class Prey(Agent):
    def __init__(self, x, y, env, id, nn = None, is_on = True):
        super().__init__(x, y, env, breed=2, channel=[0.0, 0.0, 1.0])
        self.name = f"Prey_{id}"
        self.in_danger = False
        self.is_on = is_on
        self.model_nn = nn

    def check_goal(self):
        done = False
        reward = 5.0  # Recompensa padrão para incentivar a exploração segura.
        feedback = ""
        target_detected = False
        ally_detected = False

        closest_target_distance = float('inf')
        closest_ally_distance = float('inf')

        # Procura por agente vivo, presa ou predador, mais próxima ou mantém o foco na atual.
        for agent in self.env.agents:
            # Verifica se é predador
            if agent.breed == 0 and agent.is_alive:
                distance = self.env.chebyshev_distance(self.x, self.y, agent.x, agent.y)
                if distance <= self.env.ray and (self.current_target is None or distance < closest_target_distance):
                    closest_target_distance = distance
                    self.current_target = agent
                    target_detected = True

            # Verifica se é presa com informação de predador
            elif agent.breed == 2 and agent.is_alive and agent.channel == [0, 1, 1]:
                distance = self.env.chebyshev_distance(self.x, self.y, agent.x, agent.y)
                if distance <= self.env.ray and (self.current_ally is None or distance < closest_ally_distance):
                    closest_ally_distance = distance
                    self.current_ally = agent
                    ally_detected = True

        
        # Verificação de fuga do predador
        if self.in_danger and closest_target_distance > 3:
            done = True
            self.in_danger = False
            reward += 20.0  # Recompensa máxima por escapar do perigo
            feedback = f"[PREY]: Evasao do alvo"
        
        # Critérios de Recompensa/Penalidade
        elif target_detected and 0 < closest_target_distance <= 3:
            # Penalidade por proximidade com predador
            self.in_danger = True
            if closest_target_distance == 3:
                reward -= 1.0
            elif closest_target_distance == 2:
                reward -= 3.0
            elif closest_target_distance == 1:
                reward -= 5.0
            feedback = f"[PREY]: Proximidade predador: {closest_target_distance}"

        elif ally_detected and not target_detected and 0 < closest_ally_distance <= 3:
            # Recompensa por proximidade com aliado (se não houver predador no campo de visão)
            if closest_ally_distance == 3:
                reward -= 0.1
            elif closest_ally_distance == 2:
                reward -= 0.3
            elif closest_ally_distance == 1:
                reward -= 0.5
            feedback += f"[PREY]: Proximidade aliado: {closest_ally_distance}"

        else:
            feedback += "[PREY]: Explorando mapa"


        # Atualiza channel
        for agent in self.env.agents:
            if agent.is_alive and agent.breed == 0:
                distance = self.env.chebyshev_distance(self.x, self.y, agent.x, agent.y)
                if distance <= self.env.ray:
                    self.channel = [0.0, 1.0, 1.0]
                    break
                else:
                    self.channel = [0.0, 0.0, 1.0]

        return done, reward, feedback

class Predator(Agent):
    def __init__(self, x, y, env, id, nn = None, is_on = True):
        super().__init__(x, y, env, breed=0, channel=[1.0, 0.0, 0.0])
        self.name = f"Predador_{id}"
        self.last_hunt = None
        self.is_on = is_on

    def check_goal(self):
        done = False
        reward = -0.05  # Penalidade leve para incentivar movimentação
        feedback = ""
        target_detected = False
        ally_detected = False

        closest_target_distance = float('inf')
        closest_ally_distance = float('inf')

        # Procura por agente vivo, presa ou predador, mais próxima ou mantém o foco na atual.
        for agent in self.env.agents:
            # Verifica se é presa
            if agent.is_alive and agent.breed == 2:
                distance = self.env.chebyshev_distance(self.x, self.y, agent.x, agent.y)
                if distance <= self.env.ray and (self.current_target is None or distance < closest_target_distance):
                    closest_target_distance = distance
                    self.current_target = agent
                    target_detected = True

            # Verifica se é predador com informação de presa
            elif agent.breed == 0 and agent.is_alive and agent.channel == [1.0, 1.0, 0.0]:
                distance = self.env.chebyshev_distance(self.x, self.y, agent.x, agent.y)
                if distance <= self.env.ray and (self.current_ally is None or distance < closest_ally_distance):
                    closest_ally_distance = distance
                    self.current_ally = agent
                    ally_detected = True

        # Critérios de Recompensa
        if target_detected and closest_target_distance == 0:
            done = True
            reward += 20.0  # Recompensa máxima por capturar a presa
            self.current_target.is_alive = False
            feedback = f"[PREDATOR]: Alvo capturado"
            # remover da lista esta na classe de treinamento
            self.current_target = None

        elif target_detected and 0 < closest_target_distance <= 3:
            # Recompensa por proximidade com a presa
            if closest_target_distance == 3:
                reward += 0.5
            elif closest_target_distance == 2:
                reward += 1.0
            elif closest_target_distance == 1:
                reward += 2.0
            feedback = f"[PREDATOR]: Proximidade presa: {closest_target_distance}"

        elif ally_detected and 0 < closest_ally_distance <= 3:
            # Recompensa por proximidade com aliado (que tem informação de presa)
            if closest_ally_distance == 3:
                reward += 0.1
            elif closest_ally_distance == 2:
                reward += 0.3
            elif closest_ally_distance == 1:
                reward += 0.5
            feedback = f"[PREDATOR]: Proximidade aliado: {closest_ally_distance}"

        else:
            feedback += "[PREDATOR]: Explorando mapa"

        # Atualiza channel
        for agent in self.env.agents:
            if agent.is_alive and agent.breed == 2:
                distance = self.env.chebyshev_distance(self.x, self.y, agent.x, agent.y)
                if distance <= self.env.ray:
                    self.channel = [1.0, 1.0, 0.0]
                    break
                else:
                    self.channel = [1.0, 0.0, 0.0]

        return done, reward, feedback

In [19]:
#Class: Obstacle
# ==========================================

class Obstacle:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.channel = [0.0, 1.0, 0.0]  # Cor verde para o obstáculo (RGB)

    def reset(self, env):
        self.channel = [0.0, 1.0, 0.0]

        # Escolhe uma posição aleatória no grid que esteja livre
        position = env.new_position()
        if position:
            self.x, self.y = position
        else:
            raise ValueError("Não foi possível encontrar uma posição livre para o obstáculo.")

In [20]:
#Class: Environment
# ==========================================

class Env:
    def __init__(self, sizeX, sizeY, ray=3):
        self.sizeX = sizeX
        self.sizeY = sizeY
        self.ray = ray
        self.agents = []
        self.objects = []
        self.obstacles = []  # Lista separada para obstáculos
        self.actions = 9  # Número de ações possíveis


    def new_position(self):
        # Cria uma lista de todas as posições possíveis.
        iterables = [range(self.sizeX), range(self.sizeY)]
        points = list(itertools.product(*iterables))

        # Cria uma lista das posições atuais ocupadas pelos agentes e obstáculos.
        current_positions = [(agent.x, agent.y) for agent in self.agents]
        current_positions += [(obstacle.x, obstacle.y) for obstacle in self.obstacles]

        # Filtra as posições possíveis, removendo as ocupadas.
        available_points = [point for point in points if point not in current_positions]

        # Escolhe aleatoriamente uma das posições disponíveis.
        if available_points:
            return random.choice(available_points)
        else:
            # Loga a mensagem indicando que não há posições disponíveis.
            logging.info("Não foi possível encontrar uma nova posição disponível.")
            return None

    def add_obstacle(self, obstacle):
        # Define uma posição disponível para o obstáculo
        position = self.new_position()
        if position:
            obstacle.x, obstacle.y = position  # Atribui as coordenadas ao obstáculo
            self.objects.append(obstacle)
        else:
            logging.info("Não foi possível adicionar um novo obstáculo: nenhuma posição disponível.")

    def add_agent(self, agent):
        # Define uma posição disponível para o agente
        position = self.new_position()
        if position:
            agent.x, agent.y = position  # Atribui as coordenadas ao agente
            self.objects.append(agent)
        else:
            logging.info("Não foi possível adicionar um novo agente: nenhuma posição disponível.")

    def reset(self):
        # Verifica se a lista de objetos está vazia
        if not self.objects:
            logging.info("Não é possível prosseguir: nenhum objeto foi adicionado ao ambiente.")
            return  # Interrompe o método se `self.objects` estiver vazio

        # Limpa as listas de agentes e obstáculos para reiniciar o ambiente
        self.agents = []
        self.obstacles = []

        # Embaralha a lista de objetos para variar a ordem dos elementos no ambiente
        random.shuffle(self.objects)

        # Adiciona os agentes e obstáculos à lista apropriada e os reseta
        for item in self.objects:
            item.reset(self)  # Reposiciona cada objeto no ambiente
            if isinstance(item, Agent):  # Considerando uma classe Agent
                self.agents.append(item)
            elif isinstance(item, Obstacle):  # Considerando uma classe Obstacle
                self.obstacles.append(item)

    def is_position_empty_and_valid(self, x, y):
        # Verifica se a posição está dentro dos limites do ambiente
        if x < 0 or x >= self.sizeX or y < 0 or y >= self.sizeY:
            return False  # A posição está fora dos limites do ambiente

        # Verifica se a posição está ocupada por algum agente
        for agent in self.agents:
            if agent.x == x and agent.y == y:
                return False  # A posição está ocupada

        # Verifica se a posição está ocupada por algum obstáculo
        for obstacle in self.obstacles:
            if obstacle.x == x and obstacle.y == y:
                return False  # A posição está ocupada por um obstáculo

    def get_agent_at_position(self, x, y):
        for agent in self.agents:
            if agent.x == x and agent.y == y:
                return agent
        return None

    def move_agent(self, agent, action):
        # Inicializa a penalidade padrão e os valores de penalidade
        ZERO = 0.0
        PENALIZE = -10.0
        direction = action

        # Inicializa os incrementos de movimento
        new_x, new_y = 0, 0

        # Define os incrementos de movimento com base na direção
        if direction == 0:  # Para cima
            new_y = -1
        elif direction == 1:  # Para cima e direita (diagonal)
            new_x = 1
            new_y = -1
        elif direction == 2:  # Para direita
            new_x = 1
        elif direction == 3:  # Para baixo e direita (diagonal)
            new_x = 1
            new_y = 1
        elif direction == 4:  # Para baixo
            new_y = 1
        elif direction == 5:  # Para baixo e esquerda (diagonal)
            new_x = -1
            new_y = 1
        elif direction == 6:  # Para esquerda
            new_x = -1
        elif direction == 7:  # Para cima e esquerda (diagonal)
            new_x = -1
            new_y = -1
        elif direction == 8:  # Ficar parado
            new_x = 0
            new_y = 0

        # Calcula a nova posição absoluta do agente
        target_x = agent.x + new_x
        target_y = agent.y + new_y

        # Verifica se a nova posição contém um obstáculo
        if any(obstacle.x == target_x and obstacle.y == target_y for obstacle in self.obstacles):
            print("Agente tentou ocupar um obstáculo!")
            return PENALIZE  # Penalidade por tentar ocupar a posição de um obstáculo

        # Verifica se o movimento está dentro dos limites do ambiente
        if target_x < 0 or target_x >= self.sizeX or target_y < 0 or target_y >= self.sizeY:
            print("Agente fora do limite!")
            return PENALIZE

        # Verifica se a nova posição contém outro agente
        other_agent = self.get_agent_at_position(target_x, target_y)
        if other_agent:
            # Lógica para a presa
            if agent.breed == 2:  # Presa
                print("Presa não pode ocupar a posição de outro agente!")
                return PENALIZE  # Penalidade por tentar ocupar a posição de outro agente

            # Lógica para o predador
            elif agent.breed == 0:  # Predador
                if other_agent.breed == 0:  # Outro predador na posição
                    print("Predador não pode ocupar a posição de outro predador!")
                    return PENALIZE  # Penalidade por tentar ocupar a posição de outro predador

        # Atualiza a posição do agente se o movimento for válido
        agent.x, agent.y = target_x, target_y
        return ZERO

    def render_env(self):
        a = np.zeros([self.sizeY, self.sizeX, 3])

        for agent in self.agents:
            if agent.x is not None and agent.y is not None:
                a[agent.y, agent.x, :] = agent.channel

        for obstacle in self.obstacles:
            a[obstacle.y, obstacle.x, :] = obstacle.channel  # Renderiza obstáculos

        return a

    def render_agent(self, agent):
        # Renderiza o ambiente para obter a matriz RGB atual
        a = self.render_env()

        # Calcula o tamanho do recorte com base em self.ray
        recorte_tamanho = 2 * self.ray + 1

        # Inicializa o recorte temporário com cor amarela
        recorte_temp = np.ones((recorte_tamanho, recorte_tamanho, 3)) * np.array([1.0, 1.0, 1.0])  # fora do Grid - white

        # Calcula as coordenadas do recorte dentro do ambiente
        inicio_x = agent.x - self.ray
        inicio_y = agent.y - self.ray
        fim_x = inicio_x + recorte_tamanho
        fim_y = inicio_y + recorte_tamanho

        # Calcula os limites de sobreposição entre o recorte e o ambiente
        sobreposicao_inicio_x = max(inicio_x, 0)
        sobreposicao_inicio_y = max(inicio_y, 0)
        sobreposicao_fim_x = min(fim_x, self.sizeX)
        sobreposicao_fim_y = min(fim_y, self.sizeY)

        # Calcula os índices de destino no recorte temporário
        destino_inicio_x = sobreposicao_inicio_x - inicio_x
        destino_inicio_y = sobreposicao_inicio_y - inicio_y
        destino_fim_x = destino_inicio_x + sobreposicao_fim_x - sobreposicao_inicio_x
        destino_fim_y = destino_inicio_y + sobreposicao_fim_y - sobreposicao_inicio_y

        # Copia a sobreposição do ambiente para o recorte temporário
        recorte_temp[destino_inicio_y:destino_fim_y, destino_inicio_x:destino_fim_x] = \
            a[sobreposicao_inicio_y:sobreposicao_fim_y, sobreposicao_inicio_x:sobreposicao_fim_x]

        # Pinta o elemento central do recorte de branco, ajustando a posição baseada em self.ray
        centro = self.ray
        recorte_temp[centro, centro, :] = np.array([0.5, 0.5, 0.5])  # Cinza - Destaque de Agente em treinamento

        return recorte_temp

    def population_count(self):
        """Retorna a quantidade de presas, predadores e obstáculos no ambiente."""
        predator_count = 0
        prey_count = 0

        for agent in self.agents:
            if agent.is_alive is True:
                if agent.breed == 0:
                    predator_count += 1
                elif agent.breed == 2:
                    prey_count += 1

        obstacle_count = len(self.obstacles)  # Conta o total de obstáculos na lista `self.obstacles`
        return prey_count, predator_count, obstacle_count

    def remove_agent(self, agent):
        self.agents.remove(agent)

    @staticmethod
    def chebyshev_distance(x1, y1, x2, y2):
        distance = max(abs(x2 - x1), abs(y2 - y1))
        return distance


### MODELS
----------------------------------

In [21]:
# Support
# ==========================================

lr = 0.0001
l2_regularization = 0.01

In [22]:
# Class - RADAR
# ==========================================

class ColorCombDepthwiseConv2D(Layer):
    def __init__(self, kernel_size=(7, 7), activation='relu', padding='same', **kwargs):
        super(ColorCombDepthwiseConv2D, self).__init__(**kwargs)
        self.kernel_size = kernel_size
        self.activation = activation
        self.padding = padding

        # Convoluções individuais para cores puras
        self.conv_r = Conv2D(1, kernel_size=self.kernel_size, padding=self.padding, activation=None, name="conv_r")
        self.bn_r = BatchNormalization(name="bn_r")
        
        self.conv_g = Conv2D(1, kernel_size=self.kernel_size, padding=self.padding, activation=None, name="conv_g")
        self.bn_g = BatchNormalization(name="bn_g")
        
        self.conv_b = Conv2D(1, kernel_size=self.kernel_size, padding=self.padding, activation=None, name="conv_b")
        self.bn_b = BatchNormalization(name="bn_b")

        # Convoluções para combinações específicas (Magenta, Ciano, Amarelo)
        self.conv_magenta = Conv2D(1, kernel_size=self.kernel_size, padding=self.padding, activation=None, name="conv_magenta")
        self.bn_magenta = BatchNormalization(name="bn_magenta")
        
        self.conv_cyan = Conv2D(1, kernel_size=self.kernel_size, padding=self.padding, activation=None, name="conv_cyan")
        self.bn_cyan = BatchNormalization(name="bn_cyan")
        
        self.conv_yellow = Conv2D(1, kernel_size=self.kernel_size, padding=self.padding, activation=None, name="conv_yellow")
        self.bn_yellow = BatchNormalization(name="bn_yellow")

    def call(self, inputs, training=False):
        r, g, b = tf.split(inputs, num_or_size_splits=3, axis=-1)

        # Convoluções individuais com BatchNormalization
        r_out = self.bn_r(self.conv_r(r), training=training)
        g_out = self.bn_g(self.conv_g(g), training=training)
        b_out = self.bn_b(self.conv_b(b), training=training)

        # Combinações de canais com BatchNormalization
        magenta_out = self.bn_magenta(self.conv_magenta(r + b), training=training)
        cyan_out = self.bn_cyan(self.conv_cyan(g + b), training=training)
        yellow_out = self.bn_yellow(self.conv_yellow(r + g), training=training)

        # Concatenando saídas
        outputs = Concatenate(axis=-1, name="concat_colors")([r_out, g_out, b_out, magenta_out, cyan_out, yellow_out])

        # Ativação
        outputs = Activation(self.activation, name="activation_colors")(outputs)
        return outputs


class SpatialAttentionModule(Layer):
    def __init__(self, kernel_size=4):
        super(SpatialAttentionModule, self).__init__()
        self.conv = Conv2D(1, kernel_size=kernel_size, padding='same', activation=None, name="attention_conv")
    
    def call(self, inputs):
        avg_pool = tf.reduce_mean(inputs, axis=-1, keepdims=True)
        squared_inputs = K.square(inputs)
        l2_pool = tf.sqrt(tf.reduce_mean(squared_inputs, axis=-1, keepdims=True) + 1e-6)

        concat = Concatenate(axis=-1, name="concat_attention")([avg_pool, l2_pool])
        attention_map = self.conv(concat)
        attention_map = Activation('sigmoid', name="sigmoid_attention")(attention_map)
        return Multiply(name="apply_attention")([inputs, attention_map])



In [23]:
# Model: nn_dqn
# ==========================================

class nn_dqn(tf.keras.Model):
    def __init__(self, num_actions=9, input_shape=(7, 7, 3)):
        super(nn_dqn, self).__init__()
        self.num_actions = num_actions
        self.optimizer = Adam(learning_rate=lr)
        self.loss_fn = MeanSquaredError()

        # Entrada e camadas convolucionais
        self.input_layer = InputLayer(input_shape=input_shape)

        # Camadas convolucionais
        self.conv1 = Conv2D(32, (4, 4), strides=(1, 1), activation=None, padding='same', name="conv1_layer")
        self.bn1 = BatchNormalization(name="bn1_layer")
        self.dropout_conv1 = Dropout(0.3, name="dropout_conv1_layer")  # Dropout após conv1

        self.conv2 = Conv2D(64, (3, 3), strides=(1, 1), activation=None, padding='same', name="conv2_layer")
        self.bn2 = BatchNormalization(name="bn2_layer")
        self.dropout_conv2 = Dropout(0.3, name="dropout_conv2_layer")  # Dropout após conv2

        # Flatten e camadas densas
        self.flatten = Flatten(name="flatten_layer")
        self.dropout_flatten = Dropout(0.4, name="dropout_flatten_layer")  # Dropout após Flatten

        self.dense1 = Dense(64, activation='relu', name="dense1_layer", kernel_regularizer=l2(l2_regularization))
        self.dropout_dense1 = Dropout(0.5, name="dropout_dense1_layer")  # Dropout após dense1
        
        self.dense2 = Dense(32, activation='relu', name="dense2_layer", kernel_regularizer=l2(l2_regularization))
        self.dropout_dense2 = Dropout(0.5, name="dropout_dense2_layer")  # Dropout após dense2

        self.dense_output = Dense(num_actions, activation='linear', name="dense_output_layer", kernel_regularizer=l2(l2_regularization))

    def call(self, inputs, training=False):
        x = self.input_layer(inputs)

        # Camada convolucional 1
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x, name="relu1")
        x = self.dropout_conv1(x, training=training)  # Dropout aplicado após conv1

        # Camada convolucional 2
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x, name="relu2")
        x = self.dropout_conv2(x, training=training)  # Dropout aplicado após conv2

        # Atenção espacial e Flatten
        x = self.flatten(x)
        x = self.dropout_flatten(x, training=training)  # Dropout aplicado após Flatten

        # Camadas densas
        x = self.dense1(x)
        x = self.dropout_dense1(x, training=training)  # Dropout aplicado após dense1
        x = self.dense2(x)
        x = self.dropout_dense2(x, training=training)  # Dropout aplicado após dense2

        Q_values = self.dense_output(x)
        return Q_values


    def training_step(self, batch_data):
        states, actions, targetQ = batch_data
        with tf.GradientTape() as tape:
            Q_values = self(states, training=True)
            actions_onehot = tf.one_hot(actions, self.num_actions, dtype=tf.float32)
            Q = tf.reduce_sum(Q_values * actions_onehot, axis=1)
            loss = self.loss_fn(targetQ, Q)
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return loss

    def predict_action(self, state):
        Q_values = self(state)
        return tf.argmax(Q_values, axis=1)[0].numpy()  # Retorna a ação com o maior valor Q como um número Python

    def save_model(self, file_path):
        self.save(file_path)
        print(f"Modelo salvo em: {file_path}")

    def load_model(self, file_path):
        model_loaded = tf.keras.models.load_model(file_path)
        return model_loaded
    


In [24]:
# Model: nn_per
# ==========================================

class nn_per(tf.keras.Model):
    def __init__(self, num_actions=9, input_shape=(7, 7, 3), discount_factor=0.99):
        super(nn_per, self).__init__()
        self.num_actions = num_actions
        self.discount_factor = discount_factor
        self.optimizer = Adam(learning_rate=lr)
        self.loss_fn = MeanSquaredError()

        # Entrada e camada de combinação de cores
        self.input_layer = InputLayer(input_shape=input_shape, name="input_layer")

        # Camadas convolucionais
        self.conv1 = Conv2D(32, (4, 4), strides=(1, 1), activation=None, padding='same', name="conv1_layer")
        self.bn1 = BatchNormalization(name="bn1_layer")
        self.dropout_conv1 = Dropout(0.3, name="dropout_conv1_layer")

        self.conv2 = Conv2D(64, (3, 3), strides=(1, 1), activation=None, padding='same', name="conv2_layer")
        self.bn2 = BatchNormalization(name="bn2_layer")
        self.dropout_conv2 = Dropout(0.3, name="dropout_conv2_layer")

        # Flatten e camadas densas
        self.flatten = Flatten(name="flatten_layer")
        self.dropout_flatten = Dropout(0.3, name="dropout_flatten_layer")

        self.dense1 = Dense(64, activation='relu', name="dense1_layer", kernel_regularizer=l2(l2_regularization))
        self.dropout_dense1 = Dropout(0.4, name="dropout_dense1_layer")

        self.dense2 = Dense(32, activation='relu', name="dense2_layer", kernel_regularizer=l2(l2_regularization))
        self.dropout_dense2 = Dropout(0.4, name="dropout_dense2_layer")

        self.dense_output = Dense(num_actions, activation='linear', name="dense_output_layer", kernel_regularizer=l2(l2_regularization))

    def call(self, inputs, training=False):
        x = self.input_layer(inputs)

        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x, name="relu1")
        x = self.dropout_conv1(x, training=training)  # Aplicação do Dropout após conv1

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x, name="relu2")
        x = self.dropout_conv2(x, training=training)  # Aplicação do Dropout após conv2

        x = self.flatten(x)
        x = self.dropout_flatten(x, training=training)  # Aplicação do Dropout após Flatten

        x = self.dense1(x)
        x = self.dropout_dense1(x, training=training)  # Aplicação do Dropout após dense1
        x = self.dense2(x)
        x = self.dropout_dense2(x, training=training)  # Aplicação do Dropout após dense2

        Q_values = self.dense_output(x)
        return Q_values
    

    def training_step(self, batch_data):
        states, actions, targetQ, weights = batch_data
        with tf.GradientTape() as tape:
            Q_values = self(states, training=True)
            actions_onehot = tf.one_hot(actions, self.num_actions, dtype=tf.float32)
            Q = tf.reduce_sum(Q_values * actions_onehot, axis=1)
            td_errors = targetQ - Q
            weighted_loss = tf.reduce_mean(weights * tf.square(td_errors))

        grads = tape.gradient(weighted_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        return weighted_loss  # Apenas a perda


    def predict_action(self, state):
        Q_values = self(state)
        return tf.argmax(Q_values, axis=1)[0].numpy()  # Retorna a ação com o maior valor Q como um número Python

    def save_model(self, file_path):
        self.save(file_path)
        print(f"Modelo salvo em: {file_path}")

    def load_model(self, file_path):
        model_loaded = tf.keras.models.load_model(file_path)
        return model_loaded




In [25]:
# Model: nn_dueling
# ==========================================

class nn_dueling(tf.keras.Model):
    def __init__(self, num_actions=9, input_shape=(7, 7, 3)):
        super(nn_dueling, self).__init__()
        self.num_actions = num_actions
        self.optimizer = Adam(learning_rate=lr)
        self.loss_fn = MeanSquaredError()
        
        # Entrada e camada de combinação de cores
        self.input_layer = InputLayer(input_shape=input_shape, name="input_layer")

        # Camadas convolucionais
        self.conv1 = Conv2D(32, (4, 4), strides=(1, 1), activation=None, padding='same', name="conv1_layer")
        self.bn1 = BatchNormalization(name="bn1_layer")
        self.dropout_conv1 = Dropout(0.3, name="dropout_conv1_layer")  # Dropout na primeira camada convolucional

        self.conv2 = Conv2D(64, (3, 3), strides=(1, 1), activation=None, padding='same', name="conv2_layer")
        self.bn2 = BatchNormalization(name="bn2_layer")
        self.dropout_conv2 = Dropout(0.3, name="dropout_conv2_layer")  # Dropout na segunda camada convolucional


        # Flatten
        self.flatten = Flatten(name="flatten_layer")
        self.dropout_flatten = Dropout(0.4, name="dropout_flatten_layer")  # Dropout após Flatten

        # Camadas densas compartilhadas
        self.dense_shared1 = Dense(64, activation='relu', name="shared_dense1", kernel_regularizer=l2(l2_regularization))
        self.dropout_shared = Dropout(0.4, name="dropout_shared_layer")  # Dropout na camada compartilhada

        # Rede para Valor (V)
        self.value_dense = Dense(32, activation='relu', name="value_dense", kernel_regularizer=l2(l2_regularization))
        self.dropout_value = Dropout(0.4, name="dropout_value_layer")  # Dropout na rede de valor
        self.value_output = Dense(1, activation='linear', name="value_output", kernel_regularizer=l2(l2_regularization))

        # Rede para Vantagem (A)
        self.advantage_dense = Dense(32, activation='relu', name="advantage_dense", kernel_regularizer=l2(l2_regularization))
        self.dropout_advantage = Dropout(0.4, name="dropout_advantage_layer")  # Dropout na rede de vantagem
        self.advantage_output = Dense(num_actions, activation='linear', name="advantage_output", kernel_regularizer=l2(l2_regularization))

    def call(self, inputs, training=False):
        x = self.input_layer(inputs)

        # Camada convolucional 1
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x, name="relu1")
        x = self.dropout_conv1(x, training=training)  # Dropout aplicado

        # Camada convolucional 2
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x, name="relu2")
        x = self.dropout_conv2(x, training=training)  # Dropout aplicado
        

        x = self.flatten(x)
        x = self.dropout_flatten(x, training=training)  # Dropout aplicado após Flatten

        # Camadas compartilhadas
        x = self.dense_shared1(x)
        x = self.dropout_shared(x, training=training)  # Dropout aplicado na camada compartilhada

        # Valor (V)
        v = self.value_dense(x)
        v = self.dropout_value(v, training=training)  # Dropout aplicado na rede de valor
        v = self.value_output(v)

        # Vantagem (A)
        a = self.advantage_dense(x)
        a = self.dropout_advantage(a, training=training)  # Dropout aplicado na rede de vantagem
        a = self.advantage_output(a)

        # Combina V e A para calcular Q
        q = v + (a - tf.reduce_mean(a, axis=1, keepdims=True))
        return q
    
    def training_step(self, batch_data):
        states, actions, targetQ = batch_data
        with tf.GradientTape() as tape:
            Q_values = self(states, training=True)
            actions_onehot = tf.one_hot(actions, self.num_actions, dtype=tf.float32)
            Q = tf.reduce_sum(Q_values * actions_onehot, axis=1)
            loss = self.loss_fn(targetQ, Q)
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return loss

    def predict_action(self, state):
        Q_values = self(state)
        return tf.argmax(Q_values, axis=1)[0].numpy()  # Retorna a ação com o maior valor Q como um número Python

    def save_model(self, file_path):
        self.save(file_path)
        print(f"Modelo salvo em: {file_path}")

    def load_model(self, file_path):
        model_loaded = tf.keras.models.load_model(file_path)
        return model_loaded
    



In [26]:
# Model: nn_double
# ==========================================

class nn_double(tf.keras.Model):
    def __init__(self, num_actions=9, input_shape=(7, 7, 3), discount_factor=0.99):
        super(nn_double, self).__init__()
        self.num_actions = num_actions
        self.discount_factor = discount_factor

        # Otimizador e função de perda
        self.optimizer = Adam(learning_rate=lr)
        self.loss_fn = MeanSquaredError()

        # Camada de entrada e processamento inicial de cores
        self.input_layer = InputLayer(input_shape=input_shape, name="input_layer")


        # Camadas convolucionais da rede principal
        self.conv1 = Conv2D(32, (4, 4), strides=(1, 1), activation=None, padding="same")
        self.batch_norm1 = BatchNormalization()
        self.relu1 = tf.keras.layers.ReLU()

        self.conv2 = Conv2D(64, (3, 3), strides=(1, 1), activation=None, padding="same")
        self.batch_norm2 = BatchNormalization()
        self.relu2 = tf.keras.layers.ReLU()


        # Camadas densas
        self.flatten = Flatten()
        self.dense1 = Dense(64, activation="relu", kernel_regularizer=l2(l2_regularization))
        self.dense2 = Dense(32, activation="relu", kernel_regularizer=l2(l2_regularization))
        self.output_layer = Dense(num_actions, activation="linear", kernel_regularizer=l2(l2_regularization))

        # Camadas da rede-alvo (adicionadas as mesmas modificações)
        self.target_conv1 = Conv2D(32, (4, 4), strides=(1, 1), activation=None, padding="same")
        self.target_batch_norm1 = BatchNormalization()
        self.target_relu1 = tf.keras.layers.ReLU()

        self.target_conv2 = Conv2D(64, (3, 3), strides=(1, 1), activation=None, padding="same")
        self.target_batch_norm2 = BatchNormalization()
        self.target_relu2 = tf.keras.layers.ReLU()



        self.target_flatten = Flatten()
        self.target_dense1 = Dense(64, activation="relu", kernel_regularizer=l2(l2_regularization))
        self.target_dense2 = Dense(32, activation="relu", kernel_regularizer=l2(l2_regularization))
        self.target_output_layer = Dense(num_actions, activation="linear", kernel_regularizer=l2(l2_regularization))

    def call(self, inputs, training=False):
        """Chama a rede principal para inferência."""
        x = self.input_layer(inputs)

        x = self.conv1(x)
        x = self.batch_norm1(x, training=training)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.batch_norm2(x, training=training)
        x = self.relu2(x)


        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return self.output_layer(x)

    def target_call(self, inputs, training=False):
        """Chama a rede-alvo para inferência."""
        x = self.input_layer(inputs)
        

        x = self.target_conv1(x)
        x = self.target_batch_norm1(x, training=training)
        x = self.target_relu1(x)

        x = self.target_conv2(x)
        x = self.target_batch_norm2(x, training=training)
        x = self.target_relu2(x)

        
        x = self.target_flatten(x)
        x = self.target_dense1(x)

        x = self.target_dense2(x)
        return self.target_output_layer(x)

    def update_target_network(self):
        """Sincroniza os pesos da rede principal para a rede-alvo."""
        self.target_conv1.set_weights(self.conv1.get_weights())
        self.target_batch_norm1.set_weights(self.batch_norm1.get_weights())
        self.target_conv2.set_weights(self.conv2.get_weights())
        self.target_batch_norm2.set_weights(self.batch_norm2.get_weights())
        self.target_dense1.set_weights(self.dense1.get_weights())
        self.target_dense2.set_weights(self.dense2.get_weights())
        self.target_output_layer.set_weights(self.output_layer.get_weights())

    def training_step(self, batch_data):
        """Realiza uma etapa de treinamento com Double DQN."""
        states, actions, targetQ = batch_data
        next_states, rewards, dones = targetQ
        with tf.GradientTape() as tape:
            # Predições da rede principal
            Q_values = self(states, training=True)
            actions_onehot = tf.one_hot(actions, self.num_actions, dtype=tf.float32)
            Q = tf.reduce_sum(Q_values * actions_onehot, axis=1)

            # Double DQN: Calcula o valor-alvo
            main_Q_values_next = self(next_states)  # Rede principal para selecionar a melhor ação
            next_actions = tf.argmax(main_Q_values_next, axis=1)
            target_Q_values_next = self.target_call(next_states)  # Rede-alvo para calcular Q
            target_Q = tf.reduce_sum(target_Q_values_next * tf.one_hot(next_actions, self.num_actions), axis=1)
            target_Q = rewards + (1 - dones) * self.discount_factor * target_Q

            # Calcula o loss
            loss = self.loss_fn(target_Q, Q)

        # Gradientes e atualização dos pesos
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return loss

    def predict_action(self, state):
        """Prediz a melhor ação para um dado estado."""
        Q_values = self(tf.expand_dims(state, axis=0))
        return tf.argmax(Q_values, axis=1).numpy()[0]

    def save_model(self, file_path):
        """Salva os pesos do modelo principal."""
        self.save_weights(file_path)
        print(f"Modelo salvo em: {file_path}")

    def load_model(self, file_path):
        """Carrega os pesos do modelo principal."""
        self.load_weights(file_path)
        print(f"Modelo carregado de: {file_path}")






In [27]:
# Model: nn_radar_dqn
# ==========================================

class nn_radar_dqn(tf.keras.Model):
    def __init__(self, num_actions=9, input_shape=(7, 7, 3)):
        super(nn_radar_dqn, self).__init__()
        self.num_actions = num_actions
        self.optimizer = Adam(learning_rate=lr)
        self.loss_fn = MeanSquaredError()

        # Entrada e camadas convolucionais
        self.input_layer = InputLayer(input_shape=input_shape)
        self.color_comb_layer = ColorCombDepthwiseConv2D(kernel_size=(7, 7), activation='relu', name="color_comb_layer")

        # Camadas convolucionais
        self.conv1 = Conv2D(32, (4, 4), strides=(1, 1), activation=None, padding='same', name="conv1_layer")
        self.bn1 = BatchNormalization(name="bn1_layer")
        self.dropout_conv1 = Dropout(0.3, name="dropout_conv1_layer")  # Dropout após conv1

        self.conv2 = Conv2D(64, (3, 3), strides=(1, 1), activation=None, padding='same', name="conv2_layer")
        self.bn2 = BatchNormalization(name="bn2_layer")
        self.dropout_conv2 = Dropout(0.3, name="dropout_conv2_layer")  # Dropout após conv2

        # Camada de atenção espacial
        self.spatial_attention = SpatialAttentionModule(kernel_size=4)

        # Flatten e camadas densas
        self.flatten = Flatten(name="flatten_layer")
        self.dropout_flatten = Dropout(0.4, name="dropout_flatten_layer")  # Dropout após Flatten

        self.dense1 = Dense(64, activation='relu', name="dense1_layer", kernel_regularizer=l2(l2_regularization))
        self.dropout_dense1 = Dropout(0.5, name="dropout_dense1_layer")  # Dropout após dense1
        
        self.dense2 = Dense(32, activation='relu', name="dense2_layer", kernel_regularizer=l2(l2_regularization))
        self.dropout_dense2 = Dropout(0.5, name="dropout_dense2_layer")  # Dropout após dense2

        self.dense_output = Dense(num_actions, activation='linear', name="dense_output_layer", kernel_regularizer=l2(l2_regularization))

    def call(self, inputs, training=False):
        x = self.input_layer(inputs)
        x = self.color_comb_layer(x)

        # Camada convolucional 1
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x, name="relu1")
        x = self.dropout_conv1(x, training=training)  # Dropout aplicado após conv1

        # Camada convolucional 2
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x, name="relu2")
        x = self.dropout_conv2(x, training=training)  # Dropout aplicado após conv2

        # Atenção espacial e Flatten
        x = self.spatial_attention(x)
        x = self.flatten(x)
        x = self.dropout_flatten(x, training=training)  # Dropout aplicado após Flatten

        # Camadas densas
        x = self.dense1(x)
        x = self.dropout_dense1(x, training=training)  # Dropout aplicado após dense1
        x = self.dense2(x)
        x = self.dropout_dense2(x, training=training)  # Dropout aplicado após dense2

        Q_values = self.dense_output(x)
        return Q_values


    def training_step(self, batch_data):
        states, actions, targetQ = batch_data
        with tf.GradientTape() as tape:
            Q_values = self(states, training=True)
            actions_onehot = tf.one_hot(actions, self.num_actions, dtype=tf.float32)
            Q = tf.reduce_sum(Q_values * actions_onehot, axis=1)
            loss = self.loss_fn(targetQ, Q)
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return loss

    def predict_action(self, state):
        Q_values = self(state)
        return tf.argmax(Q_values, axis=1)[0].numpy()  # Retorna a ação com o maior valor Q como um número Python

    def save_model(self, file_path):
        self.save(file_path)
        print(f"Modelo salvo em: {file_path}")

    def load_model(self, file_path):
        model_loaded = tf.keras.models.load_model(file_path)
        return model_loaded
    

In [28]:
# Model: nn_radar_per
# ==========================================

class nn_radar_per(tf.keras.Model):
    def __init__(self, num_actions=9, input_shape=(7, 7, 3), discount_factor=0.99):
        super(nn_radar_per, self).__init__()
        self.num_actions = num_actions
        self.discount_factor = discount_factor
        self.optimizer = Adam(learning_rate=lr)
        self.loss_fn = MeanSquaredError()

        # Entrada e camada de combinação de cores
        self.input_layer = InputLayer(input_shape=input_shape, name="input_layer")
        self.color_comb_layer = ColorCombDepthwiseConv2D(kernel_size=(7, 7), activation='relu', name="color_comb_layer")

        # Camadas convolucionais
        self.conv1 = Conv2D(32, (4, 4), strides=(1, 1), activation=None, padding='same', name="conv1_layer")
        self.bn1 = BatchNormalization(name="bn1_layer")
        self.dropout_conv1 = Dropout(0.3, name="dropout_conv1_layer")

        self.conv2 = Conv2D(64, (3, 3), strides=(1, 1), activation=None, padding='same', name="conv2_layer")
        self.bn2 = BatchNormalization(name="bn2_layer")
        self.dropout_conv2 = Dropout(0.3, name="dropout_conv2_layer")
        
        # Camada de atenção espacial
        self.spatial_attention = SpatialAttentionModule(kernel_size=4)

        # Flatten e camadas densas
        self.flatten = Flatten(name="flatten_layer")
        self.dropout_flatten = Dropout(0.3, name="dropout_flatten_layer")

        self.dense1 = Dense(64, activation='relu', name="dense1_layer", kernel_regularizer=l2(l2_regularization))
        self.dropout_dense1 = Dropout(0.4, name="dropout_dense1_layer")

        self.dense2 = Dense(32, activation='relu', name="dense2_layer", kernel_regularizer=l2(l2_regularization))
        self.dropout_dense2 = Dropout(0.4, name="dropout_dense2_layer")

        self.dense_output = Dense(num_actions, activation='linear', name="dense_output_layer", kernel_regularizer=l2(l2_regularization))

    def call(self, inputs, training=False):
        x = self.input_layer(inputs)
        x = self.color_comb_layer(x)

        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x, name="relu1")
        x = self.dropout_conv1(x, training=training)  # Aplicação do Dropout após conv1

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x, name="relu2")
        x = self.dropout_conv2(x, training=training)  # Aplicação do Dropout após conv2

        x = self.spatial_attention(x)
        x = self.flatten(x)
        x = self.dropout_flatten(x, training=training)  # Aplicação do Dropout após Flatten

        x = self.dense1(x)
        x = self.dropout_dense1(x, training=training)  # Aplicação do Dropout após dense1
        x = self.dense2(x)
        x = self.dropout_dense2(x, training=training)  # Aplicação do Dropout após dense2

        Q_values = self.dense_output(x)
        return Q_values
    

    def training_step(self, batch_data):
        states, actions, targetQ, weights = batch_data
        with tf.GradientTape() as tape:
            Q_values = self(states, training=True)
            actions_onehot = tf.one_hot(actions, self.num_actions, dtype=tf.float32)
            Q = tf.reduce_sum(Q_values * actions_onehot, axis=1)
            td_errors = targetQ - Q
            weighted_loss = tf.reduce_mean(weights * tf.square(td_errors))

        grads = tape.gradient(weighted_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        return weighted_loss  # Apenas a perda


    def predict_action(self, state):
        Q_values = self(state)
        return tf.argmax(Q_values, axis=1)[0].numpy()  # Retorna a ação com o maior valor Q como um número Python

    def save_model(self, file_path):
        self.save(file_path)
        print(f"Modelo salvo em: {file_path}")

    def load_model(self, file_path):
        model_loaded = tf.keras.models.load_model(file_path)
        return model_loaded




In [29]:
# Model: nn_radar_dueling
# ==========================================

class nn_radar_dueling(tf.keras.Model):
    def __init__(self, num_actions=9, input_shape=(7, 7, 3)):
        super(nn_radar_dueling, self).__init__()
        self.num_actions = num_actions
        self.optimizer = Adam(learning_rate=lr)
        self.loss_fn = MeanSquaredError()
        
        # Entrada e camada de combinação de cores
        self.input_layer = InputLayer(input_shape=input_shape, name="input_layer")
        self.color_comb_layer = ColorCombDepthwiseConv2D(kernel_size=(7, 7), activation='relu', name="color_comb_layer")

        # Camadas convolucionais
        self.conv1 = Conv2D(32, (4, 4), strides=(1, 1), activation=None, padding='same', name="conv1_layer")
        self.bn1 = BatchNormalization(name="bn1_layer")
        self.dropout_conv1 = Dropout(0.3, name="dropout_conv1_layer")  # Dropout na primeira camada convolucional

        self.conv2 = Conv2D(64, (3, 3), strides=(1, 1), activation=None, padding='same', name="conv2_layer")
        self.bn2 = BatchNormalization(name="bn2_layer")
        self.dropout_conv2 = Dropout(0.3, name="dropout_conv2_layer")  # Dropout na segunda camada convolucional
     
        # Camada de atenção espacial
        self.spatial_attention = SpatialAttentionModule(kernel_size=4)

        # Flatten
        self.flatten = Flatten(name="flatten_layer")
        self.dropout_flatten = Dropout(0.4, name="dropout_flatten_layer")  # Dropout após Flatten

        # Camadas densas compartilhadas
        self.dense_shared1 = Dense(64, activation='relu', name="shared_dense1", kernel_regularizer=l2(l2_regularization))
        self.dropout_shared = Dropout(0.4, name="dropout_shared_layer")  # Dropout na camada compartilhada

        # Rede para Valor (V)
        self.value_dense = Dense(32, activation='relu', name="value_dense", kernel_regularizer=l2(l2_regularization))
        self.dropout_value = Dropout(0.4, name="dropout_value_layer")  # Dropout na rede de valor
        self.value_output = Dense(1, activation='linear', name="value_output", kernel_regularizer=l2(l2_regularization))

        # Rede para Vantagem (A)
        self.advantage_dense = Dense(32, activation='relu', name="advantage_dense", kernel_regularizer=l2(l2_regularization))
        self.dropout_advantage = Dropout(0.4, name="dropout_advantage_layer")  # Dropout na rede de vantagem
        self.advantage_output = Dense(num_actions, activation='linear', name="advantage_output", kernel_regularizer=l2(l2_regularization))

    def call(self, inputs, training=False):
        x = self.input_layer(inputs)
        x = self.color_comb_layer(x)

        # Camada convolucional 1
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x, name="relu1")
        x = self.dropout_conv1(x, training=training)  # Dropout aplicado

        # Camada convolucional 2
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x, name="relu2")
        x = self.dropout_conv2(x, training=training)  # Dropout aplicado
        
        # Atenção espacial
        x = self.spatial_attention(x)
        x = self.flatten(x)
        x = self.dropout_flatten(x, training=training)  # Dropout aplicado após Flatten

        # Camadas compartilhadas
        x = self.dense_shared1(x)
        x = self.dropout_shared(x, training=training)  # Dropout aplicado na camada compartilhada

        # Valor (V)
        v = self.value_dense(x)
        v = self.dropout_value(v, training=training)  # Dropout aplicado na rede de valor
        v = self.value_output(v)

        # Vantagem (A)
        a = self.advantage_dense(x)
        a = self.dropout_advantage(a, training=training)  # Dropout aplicado na rede de vantagem
        a = self.advantage_output(a)

        # Combina V e A para calcular Q
        q = v + (a - tf.reduce_mean(a, axis=1, keepdims=True))
        return q
    
    def training_step(self, batch_data):
        states, actions, targetQ = batch_data
        with tf.GradientTape() as tape:
            Q_values = self(states, training=True)
            actions_onehot = tf.one_hot(actions, self.num_actions, dtype=tf.float32)
            Q = tf.reduce_sum(Q_values * actions_onehot, axis=1)
            loss = self.loss_fn(targetQ, Q)
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return loss

    def predict_action(self, state):
        Q_values = self(state)
        return tf.argmax(Q_values, axis=1)[0].numpy()  # Retorna a ação com o maior valor Q como um número Python

    def save_model(self, file_path):
        self.save(file_path)
        print(f"Modelo salvo em: {file_path}")

    def load_model(self, file_path):
        model_loaded = tf.keras.models.load_model(file_path)
        return model_loaded
    

In [30]:
# Model: nn_radar_double
# ==========================================

class nn_radar_double(tf.keras.Model):
    def __init__(self, num_actions=9, input_shape=(7, 7, 3), discount_factor=0.99):
        super(nn_radar_double, self).__init__()
        self.num_actions = num_actions
        self.discount_factor = discount_factor

        # Otimizador e função de perda
        self.optimizer = Adam(learning_rate=lr)
        self.loss_fn = MeanSquaredError()

        # Camada de entrada e processamento inicial de cores
        self.input_layer = InputLayer(input_shape=input_shape, name="input_layer")
        self.color_comb_layer = ColorCombDepthwiseConv2D(kernel_size=(7, 7), activation='relu', name="color_comb_layer")

        # Camadas convolucionais da rede principal
        self.conv1 = Conv2D(32, (4, 4), strides=(1, 1), activation=None, padding="same")
        self.batch_norm1 = BatchNormalization()
        self.relu1 = tf.keras.layers.ReLU()

        self.conv2 = Conv2D(64, (3, 3), strides=(1, 1), activation=None, padding="same")
        self.batch_norm2 = BatchNormalization()
        self.relu2 = tf.keras.layers.ReLU()

        # Módulo de atenção espacial
        self.spatial_attention = SpatialAttentionModule(kernel_size=4)

        # Camadas densas
        self.flatten = Flatten()
        self.dense1 = Dense(64, activation="relu", kernel_regularizer=l2(l2_regularization))
        self.dense2 = Dense(32, activation="relu", kernel_regularizer=l2(l2_regularization))
        self.output_layer = Dense(num_actions, activation="linear", kernel_regularizer=l2(l2_regularization))

        # Camadas da rede-alvo (adicionadas as mesmas modificações)
        self.target_color_comb_layer = ColorCombDepthwiseConv2D(kernel_size=(7, 7), activation='relu', name="target_color_comb_layer")
        self.target_conv1 = Conv2D(32, (4, 4), strides=(1, 1), activation=None, padding="same")
        self.target_batch_norm1 = BatchNormalization()
        self.target_relu1 = tf.keras.layers.ReLU()

        self.target_conv2 = Conv2D(64, (3, 3), strides=(1, 1), activation=None, padding="same")
        self.target_batch_norm2 = BatchNormalization()
        self.target_relu2 = tf.keras.layers.ReLU()

        self.target_spatial_attention = SpatialAttentionModule(kernel_size=4)

        self.target_flatten = Flatten()
        self.target_dense1 = Dense(64, activation="relu", kernel_regularizer=l2(l2_regularization))
        self.target_dense2 = Dense(32, activation="relu", kernel_regularizer=l2(l2_regularization))
        self.target_output_layer = Dense(num_actions, activation="linear", kernel_regularizer=l2(l2_regularization))

    def call(self, inputs, training=False):
        """Chama a rede principal para inferência."""
        x = self.input_layer(inputs)
        x = self.color_comb_layer(x)

        x = self.conv1(x)
        x = self.batch_norm1(x, training=training)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.batch_norm2(x, training=training)
        x = self.relu2(x)

        x = self.spatial_attention(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return self.output_layer(x)

    def target_call(self, inputs, training=False):
        """Chama a rede-alvo para inferência."""
        x = self.input_layer(inputs)
        x = self.target_color_comb_layer(x)

        x = self.target_conv1(x)
        x = self.target_batch_norm1(x, training=training)
        x = self.target_relu1(x)

        x = self.target_conv2(x)
        x = self.target_batch_norm2(x, training=training)
        x = self.target_relu2(x)

        x = self.target_spatial_attention(x)
        x = self.target_flatten(x)
        x = self.target_dense1(x)

        x = self.target_dense2(x)
        return self.target_output_layer(x)

    def update_target_network(self):
        """Sincroniza os pesos da rede principal para a rede-alvo."""
        self.target_color_comb_layer.set_weights(self.color_comb_layer.get_weights())
        self.target_conv1.set_weights(self.conv1.get_weights())
        self.target_batch_norm1.set_weights(self.batch_norm1.get_weights())
        self.target_conv2.set_weights(self.conv2.get_weights())
        self.target_batch_norm2.set_weights(self.batch_norm2.get_weights())
        self.target_dense1.set_weights(self.dense1.get_weights())
        self.target_dense2.set_weights(self.dense2.get_weights())
        self.target_output_layer.set_weights(self.output_layer.get_weights())

    def training_step(self, batch_data):
        """Realiza uma etapa de treinamento com Double DQN."""
        states, actions, targetQ = batch_data
        next_states, rewards, dones = targetQ
        with tf.GradientTape() as tape:
            # Predições da rede principal
            Q_values = self(states, training=True)
            actions_onehot = tf.one_hot(actions, self.num_actions, dtype=tf.float32)
            Q = tf.reduce_sum(Q_values * actions_onehot, axis=1)

            # Double DQN: Calcula o valor-alvo
            main_Q_values_next = self(next_states)  # Rede principal para selecionar a melhor ação
            next_actions = tf.argmax(main_Q_values_next, axis=1)
            target_Q_values_next = self.target_call(next_states)  # Rede-alvo para calcular Q
            target_Q = tf.reduce_sum(target_Q_values_next * tf.one_hot(next_actions, self.num_actions), axis=1)
            target_Q = rewards + (1 - dones) * self.discount_factor * target_Q

            # Calcula o loss
            loss = self.loss_fn(target_Q, Q)

        # Gradientes e atualização dos pesos
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return loss

    def predict_action(self, state):
        """Prediz a melhor ação para um dado estado."""
        Q_values = self(tf.expand_dims(state, axis=0))
        return tf.argmax(Q_values, axis=1).numpy()[0]

    def save_model(self, file_path):
        """Salva os pesos do modelo principal."""
        self.save_weights(file_path)
        print(f"Modelo salvo em: {file_path}")

    def load_model(self, file_path):
        """Carrega os pesos do modelo principal."""
        self.load_weights(file_path)
        print(f"Modelo carregado de: {file_path}")





### SIMULATION
--------------

In [14]:
# Support
# ==========================================

def save_sim(task_name, episode, step, tot_predator, tot_prey, reward_total, done_total, output_dir="output_logs"):
    """
    Salva informações de treinamento em arquivos separados para cada tipo de agente (prey ou predator).
    """
    # Verifica e cria o diretório, se necessário
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Itera pelos tipos de agentes ('predator' e 'prey')
    for agent_type in ['predator', 'prey']:
        # Define o nome do arquivo com base no tipo de agente
        file_name = os.path.join(output_dir, f"{task_name}_{agent_type}_simlog.txt")

        # Constrói a linha de texto a ser gravada
        line = (
            f"Episode: {episode}, Step: {step}, "
            f"Population (Prey/Predator): {tot_prey}/{tot_predator}, "
            f"Total Reward ({agent_type}): {reward_total[agent_type]:.2f}, "
            f"Total Dones ({agent_type}): {done_total[agent_type]}\n"
        )

        # Abre o arquivo no modo de adição e escreve a linha
        with open(file_name, "a") as file:
            file.write(line)

def normalize_channel(channel):

    if isinstance(channel, (list, tuple)):
        return tuple(int(value * 255) for value in channel)
    raise TypeError("O canal deve ser uma lista ou tupla de valores.")

def footprints(obstacles, agents, task_name, episode, step, grid_width, grid_height, save_obstacles=False):
    """
    Atualizado para incluir grid_width e grid_height em cada linha.
    Remove o arquivo existente se necessário.
    """
    model_dir = os.path.join(".", "final-models", "monitor")
    os.makedirs(model_dir, exist_ok=True)
    file_name = os.path.join(model_dir, f"sim_{task_name}_footprints.csv")
    header = "Episode,Step,Type,Is_Alive,R,G,B,X,Y,Grid_Width,Grid_Height\n"

    # Remove o arquivo existente se necessário
    if os.path.exists(file_name) and step == 0 and episode == 0:
        os.remove(file_name)

    # Carrega os dados existentes e inicializa os índices dos agentes
    agent_indices = {}
    existing_data = set()

    if os.path.exists(file_name):
        with open(file_name, "r") as file:
            lines = file.readlines()
            if len(lines) > 1:
                for line in lines[1:]:
                    existing_data.add(line.strip())
                    index, _, _, agent_name, *_ = line.split(",")
                    agent_indices[agent_name] = int(index)

    # Define o modo de abertura do arquivo
    file_mode = "w" if not os.path.exists(file_name) else "a"
    index = max(agent_indices.values(), default=0) + 1

    with open(file_name, file_mode) as file:
        if file_mode == "w":
            file.write(header)

        # Grava obstáculos
        if save_obstacles and step == 0:
            for obstacle in obstacles:
                normalized_channel = normalize_channel(obstacle.channel)
                line = (
                    f"{episode},{step},Obstacle,True,"
                    f"{normalized_channel[0]},{normalized_channel[1]},{normalized_channel[2]},"
                    f"{obstacle.x},{obstacle.y},{grid_width},{grid_height}\n"
                )
                if line.strip() not in existing_data:
                    file.write(line)
                    index += 1

        # Grava agentes
        for agent in agents:
            normalized_channel = normalize_channel(agent.channel)
            is_alive = "True" if agent.is_alive else "False"
            line = (
                f"{episode},{step},{agent.name},{is_alive},"
                f"{normalized_channel[0]},{normalized_channel[1]},{normalized_channel[2]},"
                f"{agent.x},{agent.y},{grid_width},{grid_height}\n"
            )
            if line.strip() not in existing_data:
                file.write(line)
                agent_indices[agent.name] = index
                index += 1

def save_log_to_file(filename, episode, step, agent, pop_prey, pop_predator, action, reward, done, nearby, feedback):
    # Formate a string como no print original
    log_entry = (f"{episode}/{step}: {agent.name}, Pop: {pop_prey}/{pop_predator}, "
                 f"Breed:{agent.breed}, Channel: {agent.channel} Position: ({agent.x}, {agent.y}), Action: {action}, "
                 f"Reward: {reward}, Done: {done}, Nearby: {nearby}, Feedback: {feedback}")
    
    # Abra o arquivo no modo append para não sobrescrever logs antigos
    with open(filename, 'a') as file:
        file.write(log_entry + '\n')  # Adicione uma nova linha após cada entrada

def get_nearby_agents_count(agent, agents_list, max_distance=3):
    """
    Conta quantos agentes estão próximos a um agente específico, dentro de um raio especificado.

    Args:
        agent (object): O agente para o qual calcular a proximidade.
        agents_list (list): Lista de agentes no ambiente.
        max_distance (int): Distância máxima para considerar como "próximo". Default é 3.

    Returns:
        int: Número de agentes dentro da distância especificada do agente fornecido.
    """
    count = 0
    
    for other_agent in agents_list:
        # Ignorar o próprio agente e agentes que não estão vivos
        if other_agent == agent or agent.breed != other_agent.breed or not other_agent.is_alive:
            continue
        
        # Calcular a distância de Chebyshev
        distance = max(abs(agent.x - other_agent.x), abs(agent.y - other_agent.y))
        
        if distance <= max_distance:
            count += 1
    
    return count



In [15]:
# SIM
# ==========================================

class Sim:
    def __init__(self, task_name, category, env, num_predators, num_preys, limit_pop_predator, limit_pop_prey, model_nn_predator, model_nn_prey, num_episodes=100, num_steps=10):
        self.directory = f"C:/Users/beLIVE/IA/DECISYS"
        self.model_dir = os.path.join(".", "final-models", "sim")
        os.makedirs(self.model_dir, exist_ok=True)
        self.category = category
        self.task_name = task_name
        self.env = env
        self.num_predators = num_predators
        self.num_preys = num_preys
        self.num_episodes = num_episodes
        self.num_steps = num_steps
        self.model_nn_predator = model_nn_predator
        self.model_nn_prey = model_nn_prey
        self.count_episodes = 0
        self.limit_pop_predator = limit_pop_predator
        self.limit_pop_prey = limit_pop_prey

    def run(self):
        for episode in range(self.num_episodes):
            self.env.reset()

            episode_rewards = {'predator': 0, 'prey': 0}
            episode_dones = {'predator': 0, 'prey': 0}

            # Define a flag para salvar os obstáculos no início do episódio
            save_obstacles_flag = True

            for step in range(self.num_steps):
                if not any(a.breed == 2 and a.is_alive for a in self.env.agents) or not any(a.breed == 0 and a.is_alive for a in self.env.agents):
                    break

                for agent in self.env.agents:
                    if not agent.is_alive:
                        continue

                    # Render state and predict action
                    state = self.env.render_agent(agent)
                    state_expanded = np.expand_dims(state, 0)
                    model = self.model_nn_prey if agent.breed == 2 else self.model_nn_predator
                    Q_values = model.predict(state_expanded, verbose=0)

                    
                    if self.category == "prqipydb":
                       action = np.argmax(Q_values[0]) if agent.breed == 0 else np.random.randint(0, 9)
                    elif self.category == "prdbpyqi":
                        action = np.argmax(Q_values[0]) if agent.breed == 2 else np.random.randint(0, 9)
                    else: 
                        action = np.argmax(Q_values[0])
                        

                    # Perform action
                    _, reward, done, feedback = agent.step(action)


                    # Atualiza os dados de recompensas e terminais
                    if agent.breed == 0:  # Predador
                        episode_rewards['predator'] += reward
                        episode_dones['predator'] += 1 if done else 0
                    elif agent.breed == 2:  # Presa
                        episode_rewards['prey'] += reward
                        episode_dones['prey'] += 1 if done else 0

                    agents_list = self.env.agents
                    nearby = get_nearby_agents_count(agent, agents_list)
                    pop_prey, pop_predator, _ = self.env.population_count()
                    # Print agent details
                    print(f"{episode}/{step}: {agent.name}, Pop: {pop_prey}/{pop_predator}, "
                          f"Breed:{agent.breed}, Color: {agent.channel}  Position: ({agent.x}, {agent.y}), Action: {action}, Reward: {reward}, Done: {done}, Nearby: {nearby}, Feedback: {feedback}")
                    save_log_to_file(self.task_name, episode, step, agent, pop_prey, pop_predator, action, reward, done, nearby, feedback)

                # Salva os footprints dos agentes a cada passo
                footprints(self.env.obstacles, self.env.agents, self.task_name, episode, step, self.env.sizeX, self.env.sizeY,save_obstacles=save_obstacles_flag)

                # Após salvar os obstáculos pela primeira vez, desativa a flag
                if save_obstacles_flag: save_obstacles_flag = False
                
            save_sim(self.task_name, episode, step, pop_predator, pop_prey, episode_rewards, episode_dones, output_dir=self.model_dir)

            



### RUN
________________________

In [152]:
# SUPORTE - Extensivo e Escasso

size = 20
num_preys = 15
num_predators = 10
num_episodes = 100
perc_obs = 0.05
num_steps = 10
v = 10

In [ ]:
# 1. Load - nn_dqn
#==========================

# Input shape para os modelos
input_shape = (7, 7, 3)
nn_model = "nn_dqn"

# Diretório e caminhos para os arquivos salvos
directory = "C:/Users/beLIVE/IA/DECISYS/final-models/last/model"
file_path_model_prey = os.path.join(directory, f'model_{nn_model}_prey.h5')
                                               
file_path_model_predator = os.path.join(directory, f'model_{nn_model}_predator.h5')

# ===========================
# Configuração do modelo de Presas
# ===========================
# Criação do modelo de presas
model_nn_prey = globals()[nn_model](num_actions=9, input_shape=input_shape)

# Inicialização do modelo de presas (rede principal e alvo)
dummy_input = tf.random.uniform((1, *input_shape))
model_nn_prey(dummy_input)  # Ativa a rede principal
#model_nn_prey.target_call(dummy_input)  # Ativa a rede-alvo
#model_nn_prey.update_target_network()  # Cria e sincroniza a rede-alvo

# Carregamento dos pesos para o modelo de presas
try:
    model_nn_prey.load_weights(file_path_model_prey)
    print("Pesos do modelo de presas carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar os pesos do modelo de presas: {e}")

# ===========================
# Configuração do modelo de Predadores
# ===========================
# Criação do modelo de predadores
model_nn_predator = globals()[nn_model](num_actions=9, input_shape=input_shape)

# Inicialização do modelo de predadores (rede principal e alvo)
model_nn_predator(dummy_input)  # Ativa a rede principal
#model_nn_predator.target_call(dummy_input)  # Ativa a rede-alvo
#model_nn_predator.update_target_network()  # Cria e sincroniza a rede-alvo

# Carregamento dos pesos para o modelo de predadores
try:
    model_nn_predator.load_weights(file_path_model_predator)
    print("Pesos do modelo de predadores carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar os pesos do modelo de predadores: {e}")

# ===========================
# Finalização
# ===========================
print("Modelos prontos para simulação!")


In [ ]:
# 2. RUN - prqipyqi
# ==========================================

category = "prqipyqi"

# Carregar os pesos dos modelos aqui, se necessário
task_name = f"sim_{nn_model}{v}_sz{size}_s{num_steps}_py{num_preys}_pd{num_predators}_o{perc_obs}_{category}"


env = Env(sizeX=size, sizeY=size, ray=3)
num_obstacle = int(env.sizeX * env.sizeX * perc_obs)

# Povoando o ambiente com obstacle
for o in range(num_obstacle):
    pos = env.new_position()
    if pos is not None:
        x, y = pos
        obstacle = Obstacle(x, y)
        env.add_obstacle(obstacle)
    else:
        print("Não foi possível encontrar uma posição nova para o obstacle.")

# Povoando o ambiente com presas
for j in range(num_preys):
    pos = env.new_position()
    if pos is not None:
        x, y = pos
        prey = Prey(x, y, env, j, model_nn_prey)
        env.add_agent(prey)
    else:
        print("Não foi possível encontrar uma posição nova para o prey.")

# Povoando o ambiente com predadores
for i in range(num_predators):
    pos = env.new_position()
    if pos is not None:
        x, y = pos
        predator = Predator(x, y, env, i, model_nn_predator)
        env.add_agent(predator)
    else:
        print("Não foi possível encontrar uma posição nova para o predator.")
limit_pop_predator = 0
limit_pop_prey = 0

# Executando a sessão de teste
sim = Sim(task_name, category, env, num_predators, num_preys, limit_pop_predator, limit_pop_prey, model_nn_predator, model_nn_prey, num_episodes, num_steps)

sim.run()


In [ ]:
# 3 - Load nn_per
#==========================

# Input shape para os modelos
input_shape = (7, 7, 3)
nn_model = "nn_per"


# Diretório e caminhos para os arquivos salvos
directory = "C:/Users/beLIVE/IA/DECISYS/final-models/last/model"
file_path_model_prey = os.path.join(directory, f'model_{nn_model}_prey.h5')
                                               
file_path_model_predator = os.path.join(directory, f'model_{nn_model}_predator.h5')

# ===========================
# Configuração do modelo de Presas
# ===========================
# Criação do modelo de presas
model_nn_prey = globals()[nn_model](num_actions=9, input_shape=input_shape)

# Inicialização do modelo de presas (rede principal e alvo)
dummy_input = tf.random.uniform((1, *input_shape))
model_nn_prey(dummy_input)  # Ativa a rede principal
#model_nn_prey.target_call(dummy_input)  # Ativa a rede-alvo
#model_nn_prey.update_target_network()  # Cria e sincroniza a rede-alvo

# Carregamento dos pesos para o modelo de presas
try:
    model_nn_prey.load_weights(file_path_model_prey)
    print("Pesos do modelo de presas carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar os pesos do modelo de presas: {e}")

# ===========================
# Configuração do modelo de Predadores
# ===========================
# Criação do modelo de predadores
model_nn_predator = globals()[nn_model](num_actions=9, input_shape=input_shape)

# Inicialização do modelo de predadores (rede principal e alvo)
model_nn_predator(dummy_input)  # Ativa a rede principal
#model_nn_predator.target_call(dummy_input)  # Ativa a rede-alvo
#model_nn_predator.update_target_network()  # Cria e sincroniza a rede-alvo

# Carregamento dos pesos para o modelo de predadores
try:
    model_nn_predator.load_weights(file_path_model_predator)
    print("Pesos do modelo de predadores carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar os pesos do modelo de predadores: {e}")

# ===========================
# Finalização
# ===========================
print("Modelos prontos para simulação!")


In [ ]:
# 4. RUN - prqipyqi
# ==========================================

category = "prqipyqi"


# Carregar os pesos dos modelos aqui, se necessário
task_name = f"sim_{nn_model}{v}_sz{size}_s{num_steps}_py{num_preys}_pd{num_predators}_o{perc_obs}_{category}"

env = Env(sizeX=size, sizeY=size, ray=3)
num_obstacle = int(env.sizeX * env.sizeX * perc_obs)

# Povoando o ambiente com obstacle
for o in range(num_obstacle):
    pos = env.new_position()
    if pos is not None:
        x, y = pos
        obstacle = Obstacle(x, y)
        env.add_obstacle(obstacle)
    else:
        print("Não foi possível encontrar uma posição nova para o obstacle.")

# Povoando o ambiente com presas
for j in range(num_preys):
    pos = env.new_position()
    if pos is not None:
        x, y = pos
        prey = Prey(x, y, env, j, model_nn_prey)
        env.add_agent(prey)
    else:
        print("Não foi possível encontrar uma posição nova para o prey.")

# Povoando o ambiente com predadores
for i in range(num_predators):
    pos = env.new_position()
    if pos is not None:
        x, y = pos
        predator = Predator(x, y, env, i, model_nn_predator)
        env.add_agent(predator)
    else:
        print("Não foi possível encontrar uma posição nova para o predator.")
limit_pop_predator = 0
limit_pop_prey = 0

# Executando a sessão de teste
sim = Sim(task_name, category, env, num_predators, num_preys, limit_pop_predator, limit_pop_prey, model_nn_predator, model_nn_prey, num_episodes, num_steps)

sim.run()


In [ ]:
# 5 - Load nn_dueling
#==========================

# Input shape para os modelos
input_shape = (7, 7, 3)
nn_model = "nn_dueling"


# Diretório e caminhos para os arquivos salvos
directory = "C:/Users/beLIVE/IA/DECISYS/final-models/last/model"
file_path_model_prey = os.path.join(directory, f'model_{nn_model}_prey.h5')
                                               
file_path_model_predator = os.path.join(directory, f'model_{nn_model}_predator.h5')

# ===========================
# Configuração do modelo de Presas
# ===========================
# Criação do modelo de presas
model_nn_prey = globals()[nn_model](num_actions=9, input_shape=input_shape)

# Inicialização do modelo de presas (rede principal e alvo)
dummy_input = tf.random.uniform((1, *input_shape))
model_nn_prey(dummy_input)  # Ativa a rede principal
#model_nn_prey.target_call(dummy_input)  # Ativa a rede-alvo
#model_nn_prey.update_target_network()  # Cria e sincroniza a rede-alvo

# Carregamento dos pesos para o modelo de presas
try:
    model_nn_prey.load_weights(file_path_model_prey)
    print("Pesos do modelo de presas carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar os pesos do modelo de presas: {e}")

# ===========================
# Configuração do modelo de Predadores
# ===========================
# Criação do modelo de predadores
model_nn_predator = globals()[nn_model](num_actions=9, input_shape=input_shape)

# Inicialização do modelo de predadores (rede principal e alvo)
model_nn_predator(dummy_input)  # Ativa a rede principal
#model_nn_predator.target_call(dummy_input)  # Ativa a rede-alvo
#model_nn_predator.update_target_network()  # Cria e sincroniza a rede-alvo

# Carregamento dos pesos para o modelo de predadores
try:
    model_nn_predator.load_weights(file_path_model_predator)
    print("Pesos do modelo de predadores carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar os pesos do modelo de predadores: {e}")

# ===========================
# Finalização
# ===========================
print("Modelos prontos para simulação!")


In [ ]:
# 6. RUN - prqipyqi
# ==========================================

category = "prqipyqi"

# Carregar os pesos dos modelos aqui, se necessário
task_name = f"sim_{nn_model}{v}_sz{size}_s{num_steps}_py{num_preys}_pd{num_predators}_o{perc_obs}_{category}"


env = Env(sizeX=size, sizeY=size, ray=3)
num_obstacle = int(env.sizeX * env.sizeX * perc_obs)

# Povoando o ambiente com obstacle
for o in range(num_obstacle):
    pos = env.new_position()
    if pos is not None:
        x, y = pos
        obstacle = Obstacle(x, y)
        env.add_obstacle(obstacle)
    else:
        print("Não foi possível encontrar uma posição nova para o obstacle.")

# Povoando o ambiente com presas
for j in range(num_preys):
    pos = env.new_position()
    if pos is not None:
        x, y = pos
        prey = Prey(x, y, env, j, model_nn_prey)
        env.add_agent(prey)
    else:
        print("Não foi possível encontrar uma posição nova para o prey.")

# Povoando o ambiente com predadores
for i in range(num_predators):
    pos = env.new_position()
    if pos is not None:
        x, y = pos
        predator = Predator(x, y, env, i, model_nn_predator)
        env.add_agent(predator)
    else:
        print("Não foi possível encontrar uma posição nova para o predator.")
limit_pop_predator = 0
limit_pop_prey = 0

# Executando a sessão de teste
sim = Sim(task_name, category, env, num_predators, num_preys, limit_pop_predator, limit_pop_prey, model_nn_predator, model_nn_prey, num_episodes, num_steps)

sim.run()


In [ ]:
# 7 - Load nn_double
#==========================

# Input shape para os modelos
input_shape = (7, 7, 3)
nn_model = "nn_double"


# Diretório e caminhos para os arquivos salvos
directory = "C:/Users/beLIVE/IA/DECISYS/final-models/last/model"
file_path_model_prey = os.path.join(directory, f'model_{nn_model}_prey.h5')
                                               
file_path_model_predator = os.path.join(directory, f'model_{nn_model}_predator.h5')

# ===========================
# Configuração do modelo de Presas
# ===========================
# Criação do modelo de presas
model_nn_prey = globals()[nn_model](num_actions=9, input_shape=input_shape)

# Inicialização do modelo de presas (rede principal e alvo)
dummy_input = tf.random.uniform((1, *input_shape))
model_nn_prey(dummy_input)  # Ativa a rede principal
model_nn_prey.target_call(dummy_input)  # Ativa a rede-alvo
model_nn_prey.update_target_network()  # Cria e sincroniza a rede-alvo

# Carregamento dos pesos para o modelo de presas
try:
    model_nn_prey.load_weights(file_path_model_prey)
    print("Pesos do modelo de presas carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar os pesos do modelo de presas: {e}")

# ===========================
# Configuração do modelo de Predadores
# ===========================
# Criação do modelo de predadores
model_nn_predator = globals()[nn_model](num_actions=9, input_shape=input_shape)

# Inicialização do modelo de predadores (rede principal e alvo)
model_nn_predator(dummy_input)  # Ativa a rede principal
model_nn_predator.target_call(dummy_input)  # Ativa a rede-alvo
model_nn_predator.update_target_network()  # Cria e sincroniza a rede-alvo

# Carregamento dos pesos para o modelo de predadores
try:
    model_nn_predator.load_weights(file_path_model_predator)
    print("Pesos do modelo de predadores carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar os pesos do modelo de predadores: {e}")

# ===========================
# Finalização
# ===========================
print("Modelos prontos para simulação!")


In [ ]:
# 8. RUN - prqipyqi
# ==========================================

category = "prqipyqi"

# Carregar os pesos dos modelos aqui, se necessário
task_name = f"sim_{nn_model}{v}_sz{size}_s{num_steps}_py{num_preys}_pd{num_predators}_o{perc_obs}_{category}"


env = Env(sizeX=size, sizeY=size, ray=3)
num_obstacle = int(env.sizeX * env.sizeX * perc_obs)

# Povoando o ambiente com obstacle
for o in range(num_obstacle):
    pos = env.new_position()
    if pos is not None:
        x, y = pos
        obstacle = Obstacle(x, y)
        env.add_obstacle(obstacle)
    else:
        print("Não foi possível encontrar uma posição nova para o obstacle.")

# Povoando o ambiente com presas
for j in range(num_preys):
    pos = env.new_position()
    if pos is not None:
        x, y = pos
        prey = Prey(x, y, env, j, model_nn_prey)
        env.add_agent(prey)
    else:
        print("Não foi possível encontrar uma posição nova para o prey.")

# Povoando o ambiente com predadores
for i in range(num_predators):
    pos = env.new_position()
    if pos is not None:
        x, y = pos
        predator = Predator(x, y, env, i, model_nn_predator)
        env.add_agent(predator)
    else:
        print("Não foi possível encontrar uma posição nova para o predator.")
limit_pop_predator = 0
limit_pop_prey = 0

# Executando a sessão de teste
sim = Sim(task_name, category, env, num_predators, num_preys, limit_pop_predator, limit_pop_prey, model_nn_predator, model_nn_prey, num_episodes, num_steps)

sim.run()
